In [5]:
# Importa as bibliotecas necessárias
import wget
import pandas as pd
import json
import datetime
from pandas.io.json import json_normalize

In [2]:
# Define os dados para download
url = 'http://plataforma.saude.gov.br/novocoronavirus/resources/scripts/'
arquivo = 'database.js'

In [ ]:
# Baixa o arquivo JS
wget.download(url + arquivo, arquivo)

In [3]:
# Converte o arquivo JavaScript para JSON
base_js = open(arquivo, "rt")
base_json = open("database.json", "wt")

for line in base_js:
	base_json.write(line.replace('var database=', ''))

base_js.close()
base_json.close()

In [6]:
# Exporta o dataframe
df = json.load(open('database.json'))
df = json_normalize(data=df['brazil'], record_path='values',meta=['date','time'])

In [7]:
# Insere o nome dos estados
indice = pd.read_csv('indice.csv')
df = pd.merge(df, indice, on='uid', how='left')

In [9]:
# Transforma números em inteiros
dados_numericos = ['suspects', 'refuses', 'confirmado', 'deads','cases']

for coluna in dados_numericos:
    df[coluna] = df[coluna].fillna(0)
    df[coluna] = df[coluna].astype(int)

In [10]:
# Exporta a base em CSV com a data da execução no nome do arquivo
agora = str(datetime.date.today()).replace(" ", "").replace(":", "-").replace(".", "-")
arquivo = 'dados/coronabr_' + agora + '.csv'
df.to_csv(arquivo, index = False)